# Christine Mayer

## clm293

## ECE 4250 Final Project 

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import matplotlib.image as mpimg
import cv2 
import scipy
import scipy.optimize
import math
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
from scipy import stats
from prettytable import PrettyTable
from scipy.interpolate import NearestNDInterpolator

Just loading up images and as in from Milestone 1

In [29]:
#milestone 1 stuff
#loading mri files:
#training 1-6, images and segmentations
img1 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_01/images/analyze/IBSR_01_ana.img')
img2 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_02/images/analyze/IBSR_02_ana.img')
img3 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_03/images/analyze/IBSR_03_ana.img')
img4 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_04/images/analyze/IBSR_04_ana.img')
img5 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_05/images/analyze/IBSR_05_ana.img')
img6 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_06/images/analyze/IBSR_06_ana.img')
seg1 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_01/segmentation/analyze/IBSR_01_seg_ana.img')
seg2 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_02/segmentation/analyze/IBSR_02_seg_ana.img')
seg3 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_03/segmentation/analyze/IBSR_03_seg_ana.img')
seg4 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_04/segmentation/analyze/IBSR_04_seg_ana.img')
seg5 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_05/segmentation/analyze/IBSR_05_seg_ana.img')
seg6 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Training/IBSR_06/segmentation/analyze/IBSR_06_seg_ana.img')

#validation 7, image and segmentation
img7 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Validation/IBSR_07/images/analyze/IBSR_07_ana.img')
seg7 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Validation/IBSR_07/segmentation/analyze/IBSR_07_seg_ana.img')
#testing 8-14, images
img8 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_08/images/analyze/IBSR_08_ana.img')
img9 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_09/images/analyze/IBSR_09_ana.img')
img10 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_10/images/analyze/IBSR_10_ana.img')
img11 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_11/images/analyze/IBSR_11_ana.img')
img12 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_12/images/analyze/IBSR_12_ana.img')
img13 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_13/images/analyze/IBSR_13_ana.img')
img14 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_14/images/analyze/IBSR_14_ana.img')
#validation 15, image and segmentation
img15 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Validation/IBSR_15/images/analyze/IBSR_15_ana.img')
seg15 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Validation/IBSR_15/segmentation/analyze/IBSR_15_seg_ana.img')
#last 2 testing, images
img16 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_16/images/analyze/IBSR_16_ana.img')
img17 = nib.load('/Users/Chrissy/Desktop/ECE4250/finalproject/ForClass/Testing/IBSR_17/images/analyze/IBSR_17_ana.img')

trainingimages=[img1,img2,img3,img4,img5,img6]
trainingseg=[seg1,seg2,seg3,seg4,seg5,seg6]
validationimages=[img7,img15]
validationseg=[seg7,seg15]
testing=[img8,img9,img10,img11,img12,img13,img14,img16,img17]

training_images_coronal=[]
for i in range(len(trainingimages)):
    slicee=trainingimages[i].get_fdata()[:, :, 64] 
    training_images_coronal.append(slicee/slicee.max())


    
training_seg_cornonal=[]
for i in range(len(trainingseg)):
    slicee=trainingseg[i].get_fdata()[:, :, 64] 
    training_seg_cornonal.append(slicee) #norm)

    
    
validation_images_coronal=[]
for i in range(len(validationimages)):
    slicee=validationimages[i].get_fdata()[:, :, 64] 
    validation_images_coronal.append(slicee/slicee.max()) #norm)

    
    
validation_seg_coronal=[]
for i in range(len(validationseg)):
    slicee=validationseg[i].get_fdata()[:, :, 64] 
    validation_seg_coronal.append(slicee)

    
    
testingcoronal=[]
for i in range(len(testing)):
    slicee=testing[i].get_fdata()[:, :, 64] 
    testingcoronal.append(slicee/slicee.max())




### Start Milestone 2 stuff!!! 

A function that takes in an input (moving) image, 4 transformation parameters (global scale, rotation, and translations along two axes) and output
(fixed image) grid size, and computes the output (moved) image. The output image has a size of the output grid size and pixel values are obtained by
resampling the input image onto the output grid via applying the geometric
transformation: a global scale multiplied with a rotation matrix, followed by
a translation. Outside-of-grid-range values can default to zero.

In [30]:
#moved with added parameters: scale x, scaley, shear x and shear y

def moved(x, scale_x, scale_y, angle, translate_x,translate_y,shear_x,shear_y):
    rad = angle*np.pi/180
    max_sz = 256
    b = np.linspace(-max_sz//2, max_sz//2, max_sz)
    bb, aa = np.array(np.meshgrid(b,b))
    
    R = np.array([[np.cos(rad), -np.sin(rad)+shear_x],[np.sin(rad)+shear_y, np.cos(rad)]])
    #Transform the coordinates, including shear into rotation matrix

   
    
    c_1 = 1/scale_y*(R[0,0]*aa + R[0,1]*bb)+translate_x
    c_2 = 1/scale_x*(R[1,0]*aa + R[1,1]*bb)-translate_y
    
    
    
    
    b_orig = np.linspace(-128, 128, 256)
    f = RegularGridInterpolator((b_orig, b_orig), x, bounds_error = False,fill_value=0)
    x_out = f(np.stack((c_1, c_2), axis=2))
    
    
    return x_out


A loss function that takes three inputs: a length-4 vector of geometric transformation parameters, a fixed image, and a moving image. The output should
be equal to the sum of squared differences between the geometrically transformed moved image and the fixed image.

In [31]:
def loss_fn(Params, img_fixed, img_moving):
   
    #inputs for moved:
    scalex=Params[0]
    scaley=Params[1]
    rotation_angle=Params[2]
    translate_x=Params[3]
    translate_y=Params[4]
    shear_x=Params[5]
    shear_y=Params[6]
    

    
    #transformation via moved function
    transformed_moved=moved(img_moving,scalex, scaley, rotation_angle, translate_x, translate_y, shear_x, shear_y)
    

    
    img_fixed=np.array(img_fixed)
    transformed_moved=np.array(transformed_moved)
    
    #sum of squared differences:
            
    summ=((img_fixed-transformed_moved)**2).sum()
    

    return summ



An optimization module that minimizes the loss function for a given input
image pair (fixed and moving). This module should return two things: the
transformed moving image and the optimal geometric transformation parameters.

In [32]:
def optimizer(fixed_image, moving_image):
    
    opt=scipy.optimize.minimize(loss_fn, [1.001,1.001,.001,.001,.001,.001,.001], args=(fixed_image, moving_image), method='BFGS', jac=None, tol=None, callback=None, options={'gtol': 1e-05, 'norm': np.inf, 'eps': 1.4901161193847656e-08, 'maxiter': None, 'disp': False, 'return_all': False})

    #print(opt)
    scalex_opt=opt.x[0]
    scaley_opt=opt.x[1]
    rot_opt=opt.x[2]
    x_opt=opt.x[3]
    y_opt=opt.x[4]
    shear_x=opt.x[5]
    shear_y=opt.x[6]
    
    return moved(moving_image, scalex_opt, scaley_opt, rot_opt, x_opt, y_opt, shear_x, shear_y), [scalex_opt, scaley_opt,rot_opt,x_opt,y_opt, shear_x, shear_y]


Part 2 -- Use your registration tool to resample each training image (moving) onto each
validation image (fixed) (i.e., you need to run 12 registration instances). Visualize
some slices of these results. You need to show that your registration works - i.e.,
plot results for before the registration and after the registration.


In [33]:
#trainingimages[0:5]
#validationimgages[0:1]

#resampled stores an array of training images onto validation images

resampled=[]
optimalparams=np.zeros((6,2,7))

for i in range(6):
    for j in range(2):
        
        #print(np.max(validation_images_coronal[j]), np.max(training_images_coronal[i]))
        img,parameters=optimizer(validation_images_coronal[j] ,training_images_coronal[i])
        
        resampled.append(np.array(img))
        #print(parameters)
        optimalparams[i][j]=[parameters[0],parameters[1],parameters[2],parameters[3], parameters[4], parameters[5], parameters[6]]
        


   

Part 3 -- Apply the registration results (optimal transformations) to resample the manual segmentations of each training subject onto the validation subject grids (use
nearest neighbor interpolation)

In [34]:
#want to use nearest neighbor interpolator instead

def moved_nearest(x, scalex, scaley, angle, translate_x,translate_y, shear_x, shear_y):
    rad = angle*np.pi/180
    max_sz = 256
    b = np.linspace(-max_sz//2, max_sz//2, max_sz)
    bb, aa = np.array(np.meshgrid(b,b))
    R = np.array([[np.cos(rad), -np.sin(rad)+shear_x],[np.sin(rad)+shear_y, np.cos(rad)]])
    #Transform the coordinates
    
    c_1 = 1/scaley*(R[0,0]*aa + R[0,1]*bb)+translate_x
    c_2 = 1/scalex*(R[1,0]*aa + R[1,1]*bb)-translate_y
    
    
    b_orig = np.linspace(-128, 128, 256)
    f = RegularGridInterpolator((b_orig, b_orig), x,method="nearest", bounds_error = False,fill_value=0)
    x_out = f(np.stack((c_1, c_2), axis=2))
    
    return x_out[:,:,0]



In [35]:
#parts 3 and 4 combined


#want new moved with nearest neighbor interp

seg_onto_val0 = [moved_nearest(training_seg_cornonal[0],optimalparams[0,0,0],optimalparams[0,0,1],optimalparams[0,0,2],optimalparams[0,0,3], optimalparams[0][0][4],optimalparams[0][0][5],optimalparams[0][0][6]),
moved_nearest(training_seg_cornonal[1],optimalparams[1][0][0],optimalparams[1][0][1],optimalparams[1][0][2],optimalparams[1][0][3],optimalparams[1][0][4],optimalparams[1][0][5],optimalparams[1][0][6]),
moved_nearest(training_seg_cornonal[2],optimalparams[2][0][0],optimalparams[2][0][1],optimalparams[2][0][2],optimalparams[2][0][3],optimalparams[2][0][4],optimalparams[2][0][5],optimalparams[2][0][6]),
moved_nearest(training_seg_cornonal[3],optimalparams[3][0][0],optimalparams[3][0][1],optimalparams[3][0][2],optimalparams[3][0][3], optimalparams[3][0][4],optimalparams[3][0][5],optimalparams[3][0][6]),
moved_nearest(training_seg_cornonal[4],optimalparams[4][0][0],optimalparams[4][0][1],optimalparams[4][0][2],optimalparams[4][0][3], optimalparams[4][0][4],optimalparams[4][0][5],optimalparams[4][0][6]),
moved_nearest(training_seg_cornonal[5],optimalparams[5][0][0],optimalparams[5][0][1],optimalparams[5][0][2],optimalparams[5][0][3], optimalparams[5][0][4],optimalparams[5][0][5],optimalparams[5][0][6])]

seg_onto_val1 = [moved_nearest(training_seg_cornonal[0],optimalparams[0][1][0],optimalparams[0][1][1],optimalparams[0][1][2],optimalparams[0][1][3], optimalparams[0][1][4],optimalparams[0][1][5],optimalparams[0][1][6]),
moved_nearest(training_seg_cornonal[1],optimalparams[1][1][0],optimalparams[1][1][1],optimalparams[1][1][2],optimalparams[1][1][3], optimalparams[1][1][4],optimalparams[1][1][5],optimalparams[1][1][6]),
moved_nearest(training_seg_cornonal[2],optimalparams[2][1][0],optimalparams[2][1][1],optimalparams[2][1][2],optimalparams[2][1][3], optimalparams[2][1][4],optimalparams[2][1][5],optimalparams[2][1][6]),
moved_nearest(training_seg_cornonal[3],optimalparams[3][1][0],optimalparams[3][1][1],optimalparams[3][1][2],optimalparams[3][1][3], optimalparams[3][1][4],optimalparams[3][1][5],optimalparams[3][1][6]),
moved_nearest(training_seg_cornonal[4],optimalparams[4][1][0],optimalparams[4][1][1],optimalparams[4][1][2],optimalparams[4][1][3], optimalparams[4][1][4],optimalparams[4][1][5],optimalparams[4][1][6]),
moved_nearest(training_seg_cornonal[5],optimalparams[5][1][0],optimalparams[5][1][1],optimalparams[5][1][2],optimalparams[5][1][3], optimalparams[5][1][4],optimalparams[5][1][5],optimalparams[5][1][6])]



#here is part 4!

grid0=np.zeros((256,256,6))
grid1=np.zeros((256,256,6))



#"stacking" each segmentation...

for i in range(6):
    for m in range(256):
        for n in range(256):
            
            grid0[m][n][i]=seg_onto_val0[i][m][n]
            grid1[m][n][i]=seg_onto_val1[i][m][n]
        
    

#now want to find most frequent training label for each pixel across each of 6 images 

segmentation0=np.zeros((256,256))
segmentation1=np.zeros((256,256))

for m in range(256):
    for n in range(256):
        mode = stats.mode(grid0[m][n])[0]
        
        segmentation0[m][n]=mode[0]
        
        mode = stats.mode(grid1[m][n])[0]
        segmentation1[m][n]=mode[0]



/anaconda3/lib/python3.7/site-packages/scipy/interpolate/interpolate.py:2514: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.values[idx_res]


Part 4 -- For every pixel on validation subject grid, compute the most frequent training
label – this is called majority voting based label fusion. You can implement
any tie-break strategy you want. This is a crude segmentation of the validation
subjects.

### Jacarrd Computation:

Write a function that computes the Jaccard overlap index for a given region of
interest (ROI) between an input manual segmentation and an automatic segmentation. The Jaccard index is defined as the ratio between the area of the intersection and the area of the union, where the intersection and union are defined
with respect to the manual segmentation and an automatic segmentation.

In [ ]:
#Jacarrd=abs(intersection/union)
     

def Jacarrd(auto_segmentation, manual_segmentation,roi):
    
   
    auto_segmentation=np.array(auto_segmentation)

    manual_segmentation=np.array(manual_segmentation)[:,:,0]
    
    #rescale:
    auto_segmentation=auto_segmentation*manual_segmentation.max()/auto_segmentation.max()
    
    
    numerator=0
    denominator=0 
    
    for i in range(256):
        for j in range(256):
            #if auto_segmentation[i][j]!=0:
               # print(auto_segmentation[i][j])
            
            if (manual_segmentation[i][j])==roi and (int(auto_segmentation[i][j])==roi):
                numerator=numerator+1
            
                
            if (manual_segmentation[i][j]==roi) or (int(auto_segmentation[i][j])==roi):
                denominator=denominator+1

    if denominator==0:
        return 'divide by zero'
    
    return numerator/denominator



Compute the Jaccard index for your automatic validation subject segmentations.
Compile these in a table and print. Only consider following regions of interest
(both left and right): Cerebral-White-Matter and Cerebral-Cortex.

In [ ]:
#roi's:
#  Left-Cerebral-White-Matter   2 
#  Right-Cerebral-White-Matter  41       
#  Left-Cerebral-Cortex         3        
#  Right-Cerebral-Cortex        42    


lcwm0=Jacarrd(segmentation0,validation_seg_coronal[0], 2)
lcwm1=Jacarrd(segmentation1,validation_seg_coronal[1] ,2)

rcwm0=Jacarrd(segmentation0,validation_seg_coronal[0],41)
rcwm1=Jacarrd(segmentation1,validation_seg_coronal[1] ,41)

lcc0=Jacarrd(segmentation0,validation_seg_coronal[0], 3)
lcc1=Jacarrd(segmentation1,validation_seg_coronal[1] ,3)

rcc0=Jacarrd(segmentation0,validation_seg_coronal[0], 42)
rcc1=Jacarrd(segmentation1,validation_seg_coronal[1] ,42)


x = PrettyTable()

x.field_names = ["Region Of Interest", "Segmentation 0 Jaccard", "Segmentation 1 Jaccard"]

x.add_row(["Left-Cerebral-White-Matter", lcwm0, lcwm1])
x.add_row(["Right-Cerebral-White-Matter", rcwm0, rcwm1])
x.add_row(["Left-Cerebral-Cortex ", lcc0, lcc1])
x.add_row(["Right-Cerebral-Cortex", rcc0, rcc1])

print('Table of Jaccard Indexes')
print(x)
print('average:')
print((lcwm0+lcwm1+rcwm0+rcwm1+lcc0+lcc1+rcc0+rcc1)/8)
print('average - outliers:')
print((lcwm0+lcwm1+rcwm1+lcc0+lcc1+rcc1)/6)

# Testing Images

Now all final optimizations...

## Going to:

map 6 training segs PLUS 2 validation segs onto each of the testing images
    optimizer(fixed=testing, moving=training_segs)
    now have 8 registered segmentations
    get my final segmenation of that testing image by most frequent label of those 8!

In [ ]:
def final_seg(testing_img):
    
    
    optimalparams=[optimizer(testing_img,training_images_coronal[0][:,:,0])[1],optimizer(testing_img,training_images_coronal[1][:,:,0])[1],optimizer(testing_img,training_images_coronal[2][:,:,0])[1],optimizer(testing_img,training_images_coronal[3][:,:,0])[1],optimizer(testing_img,training_images_coronal[4][:,:,0])[1],optimizer(testing_img,training_images_coronal[5][:,:,0])[1], optimizer(testing_img, validation_images_coronal[0][:,:,0])[1], optimizer(testing_img, validation_images_coronal[1][:,:,0])[1]]

    
    stacked=[moved_nearest(training_seg_cornonal[0],optimalparams[0][0],optimalparams[0][1],optimalparams[0][2],optimalparams[0][3], optimalparams[0][4],optimalparams[0][5],optimalparams[0][6]),
moved_nearest(training_seg_cornonal[1],optimalparams[1][0],optimalparams[1][1],optimalparams[1][2],optimalparams[1][3],optimalparams[1][4],optimalparams[1][5],optimalparams[1][6]),
moved_nearest(training_seg_cornonal[2],optimalparams[2][0],optimalparams[2][1],optimalparams[2][2],optimalparams[2][3],optimalparams[2][4],optimalparams[2][5],optimalparams[2][6]),
moved_nearest(training_seg_cornonal[3],optimalparams[3][0],optimalparams[3][1],optimalparams[3][2],optimalparams[3][3], optimalparams[3][4],optimalparams[3][5],optimalparams[3][6]),
moved_nearest(training_seg_cornonal[4],optimalparams[4][0],optimalparams[4][1],optimalparams[4][2],optimalparams[4][3], optimalparams[4][4],optimalparams[4][5],optimalparams[4][6]),
moved_nearest(training_seg_cornonal[5],optimalparams[5][0],optimalparams[5][1],optimalparams[5][2],optimalparams[5][3], optimalparams[5][4],optimalparams[5][5],optimalparams[5][6]),
moved_nearest(validation_seg_coronal[0], optimalparams[6][0],optimalparams[6][1], optimalparams[6][2],optimalparams[6][3],optimalparams[6][4],optimalparams[6][5],optimalparams[6][6]), 
moved_nearest(validation_seg_coronal[1], optimalparams[7][0],optimalparams[7][1], optimalparams[7][2],optimalparams[7][3],optimalparams[7][4],optimalparams[7][5],optimalparams[7][6])]

    mftl=np.zeros((256,256))
    grid=np.zeros((256,256,8))
               
    for m in range(256):
        for n in range(256):
            for i in range(8):
                grid[m][n][i]=stacked[i][m][n]
                
   
    for m in range(256):
        for n in range(256):
            mode = stats.mode(grid[m][n])[0]
            mftl[m][n]=mode
            
    #most frequent training label, same as before^    
    
                   
    return mftl, stacked


In [ ]:
myseg, stacked = final_seg(testingcoronal[0][:,:,0])
print('1!!')
myseg1, stacked1 = final_seg(testingcoronal[1][:,:,0])
print('2!!!!')
myseg2, stacked2 = final_seg(testingcoronal[2][:,:,0])
print('3!!!!')
myseg3, stacked3 = final_seg(testingcoronal[3][:,:,0])
print('4!!!!')
myseg4, stacked4 = final_seg(testingcoronal[4][:,:,0])
print('5!!!!')
myseg5, stacked5 = final_seg(testingcoronal[5][:,:,0])
print('6!!!!')
myseg6, stacked6 = final_seg(testingcoronal[6][:,:,0])
print('7!!!!')
myseg7, stacked7 = final_seg(testingcoronal[7][:,:,0])
print('8!!!!')
myseg8, stacked8 = final_seg(testingcoronal[8][:,:,0])
print('9!!!!')

In [ ]:
'''Binary mask of auto segmentation, ROI=1, else=0'''

def get_binary(seg, roi):
    binary=np.zeros((256,256))
    for m in range(256):
        for n in range(256):
            if round(seg[m][n])==roi:
                binary[m][n]=1
    return binary



In [ ]:
#test 8
b8_2=get_binary(myseg, 2)
b8_3=get_binary(myseg, 3)
b8_41=get_binary(myseg, 41)
b8_42=get_binary(myseg, 42)

#test 9
b9_2=get_binary(myseg1, 2)
b9_3=get_binary(myseg1, 3)
b9_41=get_binary(myseg1, 41)
b9_42=get_binary(myseg1, 42)

#test 9
b10_2=get_binary(myseg2, 2)
b10_3=get_binary(myseg2, 3)
b10_41=get_binary(myseg2, 41)
b10_42=get_binary(myseg1, 42)

#test 11
b11_2=get_binary(myseg3, 2)
b11_3=get_binary(myseg3, 3)
b11_41=get_binary(myseg3, 41)
b11_42=get_binary(myseg3, 42)

#test 12
b12_2=get_binary(myseg4, 2)
b12_3=get_binary(myseg4, 3)
b12_41=get_binary(myseg4, 41)
b12_42=get_binary(myseg4, 42)

#test 13
b13_2=get_binary(myseg5, 2)
b13_3=get_binary(myseg5, 3)
b13_41=get_binary(myseg5, 41)
b13_42=get_binary(myseg5, 42)

#test 14
b14_2=get_binary(myseg6, 2)
b14_3=get_binary(myseg6, 3)
b14_41=get_binary(myseg6, 41)
b14_42=get_binary(myseg6, 42)


#test 16
b16_2=get_binary(myseg7, 2)
b16_3=get_binary(myseg7, 3)
b16_41=get_binary(myseg7, 41)
b16_42=get_binary(myseg7, 42)


#test 17
b17_2=get_binary(myseg8, 2)
b17_3=get_binary(myseg8, 3)
b17_41=get_binary(myseg8, 41)
b17_42=get_binary(myseg8, 42)



In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
for i in [b8_2,b8_3,b8_41,b8_42]:
    print(rle_encode(i))


In [ ]:
for i in [b9_2,b9_3,b9_41,b9_42]:
    print(rle_encode(i))


In [ ]:
for i in [b10_2,b10_3,b10_41,b10_42]:
    print(rle_encode(i))


In [ ]:
for i in [b11_2,b11_3,b11_41,b11_42]:
    print(rle_encode(i))


In [ ]:
for i in [b12_2,b12_3,b12_41,b12_42]:
    print(rle_encode(i))


In [ ]:
for i in [b13_2,b13_3,b13_41,b13_42]:
    print(rle_encode(i))


In [ ]:
for i in [b14_2,b14_3,b14_41,b14_42]:
    print(rle_encode(i))


In [ ]:
for i in [b16_2,b16_3,b16_41,b16_42]:
    print(rle_encode(i))


In [ ]:
for i in [b17_2,b17_3,b17_41,b17_42]:
    print(rle_encode(i))